In [16]:
import nltk
import openai
import anthropic
import google.generativeai as genai

/Users/emilyvo/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
import json
import pandas as pd
import re
import time
import random

In [18]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# import torch
import numpy as np
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer

Loading Dataset of Reviews

In [10]:
with open('dataset_goodreads_review.json', 'r') as f:
    data = json.load(f)
df = pd.json_normalize(data)

# title in bookUrl

In [29]:
def remove_html_tags(text):
    clean_text = re.sub(r'<br\s*/?>\s*<br\s*/?>', '', text)
    clean_text = re.sub(r'<br\s*/?>', '', text)  
    clean_text = re.sub(r'<.*?>', '', clean_text) 
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()  # cleaning up extra spaces
    return clean_text

df['cleaned_text'] = df['text'].apply(remove_html_tags)
print(df['cleaned_text'].head())

def preprocess(text):
    text = text.lower()  # lowercase
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    tokens = word_tokenize(text) 
    stop_words = set(stopwords.words('english'))  
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]  
    return tokens

df['cleaned_reviews'] = df['text'].apply(preprocess)

0    "He doesn't seem very impressed," Cimorene com...
1    Charming and cute. Nostalgia and a sale on the...
2    The scholarly princcesses, the very sensible d...
3    A playful, endearing, light, quick middle-grad...
4    4.5/5 starsThis is my 4th time reading this. S...
Name: cleaned_text, dtype: object


AI Model Review Evaluation
Push those same 30 books and their respective reviews into the models
Ask two questions
“Can you identify any female characters based on the reviews?”
“What is the overall sentiment that reviewers have towards that female character?”


In [25]:
with open("human_eval.json", "r", encoding="utf-8") as f:
    human_eval_data = json.load(f)
def make_slug(title):
    title = title.strip()
    title = title.replace("’", "")  # Remove smart apostrophes
    title = re.sub(r'[^\w\s]', '', title)  # Remove punctuation
    return title.replace(" ", "_")

target_books = [make_slug(entry["Book Title"]) for entry in human_eval_data]
print(target_books)


['Dealing_with_Dragons', 'Homeland', 'Outlander', 'Tam_Lin', 'Beauty', 'The_tale_of_the_body_thief', 'Parable_of_the_Sower', 'The_Last_Wish', 'Guilty_Pleasures', 'Wizards_First_Rule', 'The_Forest_House', 'Practical_Magic', 'Harry_Potter_and_the_Sorcerers_Stone', 'Daughter_of_the_Forest', 'Dead_Until_Dark', 'A_Great_and_Terrible_Beauty', 'Furies_of_Calderon', 'The_Lies_of_Locke_Lamora', 'City_of_Bones_The_Mortal_Instruments', 'Physik', 'Fallen_Fallen', 'I_Shall_Wear_Midnight', 'The_Book_of_Life', 'All_the_Birds_in_the_Sky', 'An_Excess_Male', 'The_Poppy_War', 'Gods_of_Jade_and_Shadow', 'The_Invisible_Life_of_Addie_LaRue', 'Legends__Lattes', 'Phantasma']


In [26]:
selected_books_df = df[df['bookUrl'].apply(lambda url: any(slug in url for slug in target_books))].copy()

14


In [50]:
# --- SETUP: Load Human-Eval JSON ---
with open("human_eval.json", "r", encoding="utf-8") as f:
    human_eval_data = json.load(f)

def make_slug(title):
    title = title.strip()
    title = title.replace("’", "")
    title = re.sub(r'[^\w\s]', '', title)
    return title.replace(" ", "_")

target_books = [make_slug(entry["Book Title"]) for entry in human_eval_data]

# --- FILTER: Only 30 books from human eval ---
selected_books_df = df[df['bookUrl'].apply(lambda url: any(slug in url for slug in target_books))].copy()

# --- GROUP: By book URL ---
grouped_reviews = selected_books_df.groupby("bookUrl")["cleaned_reviews"].apply(list)

# --- PROMPT BUILDER ---
def build_prompt(reviews):
    # Flatten the list of lists into a list of strings
    flat_reviews = [" ".join(r) if isinstance(r, list) else r for r in reviews]
    text = "\n\n".join(flat_reviews[:20])
    
    return f"""Here are 50 Goodreads reviews for a novel:

{text}

Questions:
1. Can you identify any female characters based on the reviews?
2. What is the overall sentiment that reviewers have towards that female character (or characters)?
Please answer both questions clearly and concisely.
"""
# --- QUERY FUNCTIONS (real model versions assumed already set up) ---
def query_gpt(prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=800
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"GPT Error: {e}"

def query_claude(prompt):
    try:
        response = anthropic_client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=800,
            temperature=0.3,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.content[0].text.strip()
    except Exception as e:
        return f"Claude Error: {e}"

def query_gemini(prompt):
    try:
        model = genai.GenerativeModel('models/gemini-1.5-pro')
        response = model.generate_content(prompt)
        time.sleep(60)
        return response.text.strip()
    except Exception as e:
        return f"Gemini Error: {e}"

def query_gemini_with_backoff(prompt, max_retries=5):
    delay = 10  # initial delay in seconds
    for attempt in range(max_retries):
        try:
            response = gemini_model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            # Check for specific rate limit error
            error_msg = str(e)
            if "429" in error_msg or "quota" in error_msg.lower():
                print(f"Gemini rate limit hit. Waiting {delay} seconds before retrying...")
                time.sleep(delay)
                delay *= 2  # exponential backoff
                delay += random.uniform(0, 3)  # jitter to avoid collision
            else:
                print(f"Gemini error (non-retryable): {e}")
                return f"Gemini Error: {e}"
    return "Gemini Error: Max retries exceeded"


In [54]:
gemini_model = genai.GenerativeModel("models/gemini-1.5-pro")

results = []

for book_url, reviews in grouped_reviews.items():
    prompt = build_prompt(reviews)

    print(f"Processing: {book_url}")

    gpt_response = query_gpt(prompt)
    claude_response = query_claude(prompt)
    gemini_response = query_gemini_with_backoff(prompt)

    results.append({
        "book_url": book_url,
        "gpt_response": gpt_response,
        "claude_response": claude_response,
        "gemini_response": gemini_response
    })


Processing: https://www.goodreads.com/book/show/10964.Outlander
Processing: https://www.goodreads.com/book/show/13928.Daughter_of_the_Forest
Processing: https://www.goodreads.com/book/show/150739.Dealing_with_Dragons 
Processing: https://www.goodreads.com/book/show/18892.The_Forest_House
Gemini rate limit hit. Waiting 10 seconds before retrying...
Gemini rate limit hit. Waiting 22.12821097943156 seconds before retrying...
Gemini rate limit hit. Waiting 45.04867016129845 seconds before retrying...
Processing: https://www.goodreads.com/book/show/22896.Practical_Magic 
Processing: https://www.goodreads.com/book/show/29396.Furies_of_Calderon 
Processing: https://www.goodreads.com/book/show/301082.Dead_Until_Dark 
Gemini rate limit hit. Waiting 10 seconds before retrying...
Processing: https://www.goodreads.com/book/show/30281.Guilty_Pleasures
Processing: https://www.goodreads.com/book/show/355916.Physik
Processing: https://www.goodreads.com/book/show/3682.A_Great_and_Terrible_Beauty 
Gemin

In [55]:
results_df = pd.DataFrame(results)
results_df["book_title"] = results_df["book_url"].apply(lambda url: url.split("/")[-1].replace("_", " "))
# save to CSV
results_df.to_csv("ai_model_review_evaluation.csv", index=False)

In [57]:
| Book Title                         | Positivity (Human) | GPT Score | Claude Score | Gemini Score | Best Model |
|-----------------------------------|---------------------|-----------|--------------|---------------|------------|
| Outlander                         | 30                  | 29        | 32           | 26            | GPT        |
| Dealing with Dragons              | 41                  | 38        | 40           | 37            | Claude     |
| Tam Lin                           | 20                  | 25        | 21           | 19            | Claude     |
| Beauty                            | 24                  | 30        | 26           | 24            | Gemini     |
| The Tale of the Body Thief       | 38                  | 37        | 39           | 38            | Gemini     |
| Parable of the Sower             | 33                  | 30        | 32           | 33            | Gemini     |
| The Last Wish                    | 34                  | 38        | 34           | 36            | Claude     |
| Guilty Pleasures                 | 28                  | 35        | 30           | 28            | Gemini     |
| Practical Magic                  | 28                  | 33        | 29           | 27            | Gemini     |
| Daughter of the Forest           | 40                  | 44        | 39           | 42            | Claude     |
| Harry Potter and the Sorcerer’s Stone | 31             | 33        | 30           | 29            | Claude     |
| A Great and Terrible Beauty      | 21                  | 27        | 22           | 21            | Gemini     |
| The Book of Life                 | 17                  | 19        | 20           | 16            | Gemini     |
| All the Birds in the Sky         | 24                  | 25        | 24           | 23            | Claude     |
| An Excess Male                   | 25                  | 23        | 27           | 25            | Gemini     |

SyntaxError: invalid character '’' (U+2019) (830168242.py, line 13)

Setting up GPT, Claude, and Gemini

In [ ]:
def query_gpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",  # or "gpt-3.5-turbo" if you want faster/cheaper
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=1000  # Adjust based on how long you want the answer
    )
    return response['choices'][0]['message']['content'].strip()


In [ ]:
def query_claude(prompt):
    response = anthropic_client.messages.create(
        model="claude-3-sonnet-20240229",  # or "claude-3-haiku-20240307" for faster/cheaper
        max_tokens=1000,
        temperature=0.2,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.content[0].text.strip()


In [ ]:
def query_gemini(prompt):
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    return response.text.strip()


AI Model Selection
For each book, we compare the AI models’ outputs to our human evaluation
Whichever has the highest similarity to our recorded answers is the one we use for the rest of our project

In [56]:
with open("human_eval.json", "r") as f:
    human_eval_data = json.load(f)

human_df = pd.DataFrame(human_eval_data)
human_df['Book Title'] = human_df['Book Title'].str.strip()
human_df['Positivity'] = human_df['Positivity'].str.extract(r'(\d+)').astype(float)


In [1]:
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

### Average Sentiment by Century

In [11]:
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Base directory
base_dir = "/Users/emilyvo/info4940/books"
sentiment_data = {'20th': [], '21st': []}

# Chunk size (words)
CHUNK_SIZE = 500

def get_chunked_sentiment(text):
    words = text.split()
    chunks = [' '.join(words[i:i+CHUNK_SIZE]) for i in range(0, len(words), CHUNK_SIZE)]
    if not chunks:
        return 0
    scores = [sid.polarity_scores(chunk)['compound'] for chunk in chunks]
    return sum(scores) / len(scores)

# Loop through folders
for century_folder in ['20th', '21st']:
    folder_path = os.path.join(base_dir, century_folder)
    print(f"\n Processing folder: {folder_path}")

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            print(f"  Reading file: {filename}")

            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read()
                    score = get_chunked_sentiment(text)
                    sentiment_data[century_folder].append(score)
                    print(f"    Avg sentiment score from chunks: {score:.4f}")
            except Exception as e:
                print(f"    Error reading {file_path}: {e}")

# Final results
print("\n=== Average Sentiment Scores ===")
for century, scores in sentiment_data.items():
    avg = sum(scores) / len(scores) if scores else 0
    print(f"{ century} century avg sentiment: {round(avg, 3)} from {len(scores)} books")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/emilyvo/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!



 Processing folder: /Users/emilyvo/info4940/books/20th
  Reading file: Harry_Potter_Chamber_of_Secrets_JK_Rowling.txt
    Avg sentiment score from chunks: 0.0437
  Reading file: A_Game_of_Thrones_George_R_R_Martin.txt
    Avg sentiment score from chunks: 0.0326
  Reading file: Ella_Enchanted__Gail_Carson_Levine.txt
    Avg sentiment score from chunks: 0.5349
  Reading file: Parable_of_the_Sower_-_Octavia_E_Butler.txt
    Avg sentiment score from chunks: -0.3198
  Reading file: Alice_Hoffman_Practical_Magic.txt
    Avg sentiment score from chunks: 0.1888
  Reading file: Daughter_Of_The_Forest_Juliet_Marillier.txt
    Avg sentiment score from chunks: 0.2037
  Reading file: Dealing-With-Dragons--RA-Salvatore.txt
    Avg sentiment score from chunks: 0.4617
  Reading file: Brown_Girl_In_the_Ring_Nalo_Hopkinson.txt
    Avg sentiment score from chunks: -0.1027
  Reading file: Outlander_Diana_Gabaldon.txt
    Avg sentiment score from chunks: 0.3074
  Reading file: Daughter_of_the_Blood_Anne_B

In [20]:

# Analyzing Parable of the Sower Example
with open("Parable_of_the_Sower.txt", 'r', encoding='utf-8') as file:
    full_text = file.read()
paragraphs = [p.strip() for p in full_text.split('\n') if len(p.strip()) > 100]
lauren_passages = paragraphs[:20]
results = []

for i, passage in enumerate(lauren_passages):
    print(f"Processing passage {i + 1}/{len(lauren_passages)}...")

    prompt = (
        f"In the following passage from *Parable of the Sower*, how is Lauren portrayed?\n"
        f"Does she appear empowered, independent, passive, emotional, objectified, or complex?\n"
        f"Please interpret tone, traits, and characterization.\n\n"
        f"Passage:\n{passage}\n\nAnswer:"
    )

    try:
        response = client.messages.create(
            model="claude-3-opus-20240229",
            max_tokens=500,
            temperature=0.7,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )

        answer = response.content[0].text.strip()
        results.append({'passage': passage, 'interpretation': answer})

    except Exception as e:
        print(f"Error on passage {i + 1}: {e}")
        results.append({'passage': passage, 'interpretation': f"Error: {e}"})

# Display results
import pandas as pd
df_claude_analysis = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
print(df_claude_analysis)


Processing passage 1/20...
Processing passage 2/20...
Processing passage 3/20...
Processing passage 4/20...
Processing passage 5/20...
Processing passage 6/20...
Processing passage 7/20...
Processing passage 8/20...
Processing passage 9/20...
Processing passage 10/20...
Processing passage 11/20...
Processing passage 12/20...
Processing passage 13/20...
Processing passage 14/20...
Processing passage 15/20...
Processing passage 16/20...
Processing passage 17/20...
Processing passage 18/20...
Processing passage 19/20...
Processing passage 20/20...
                                                                                                                                                            passage  \
0                                                          This book is a work of fiction. Names, characters, places, and incidents are the product of the author’s   
1                                                            imagination or are used fictitiously. Any resemblance 

In [21]:
# combining All Summaries
combined_summaries = "\n\n".join([f"Summary {i+1}:\n{s}" for i, s in enumerate(results)])
final_prompt = f"""Here are summaries of multiple passages from *Parable of the Sower* about Lauren:\n\n{combined_summaries}\n\nBased on these summaries, how is Lauren portrayed overall? What recurring traits, tones, or character arcs appear across the novel?"""

try:
    final_message = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=500,
        temperature=0.5,
        messages=[{"role": "user", "content": final_prompt}]
    )
    final_summary = final_message.content[0].text.strip()
    print("\n=== Final Overall Character Interpretation ===\n")
    print(final_summary)

except Exception as e:
    print(f"Error during final synthesis: {e}")


=== Final Overall Character Interpretation ===

Based on the provided summaries, Lauren is consistently portrayed as an empowered, independent, and complex character throughout the novel "Parable of the Sower."

Recurring traits:

1. Observant and analytical: Lauren is highly perceptive, noting details about her surroundings, such as the living conditions in her neighborhood, the weapons carried by security guards, and the emotional atmosphere of the places she travels through.

2. Independent thinker: Lauren forms her own opinions, makes her own decisions, and is not afraid to express her thoughts. She critically analyzes situations and does not rely on others to shape her perspectives.

3. Resilient and adaptable: Despite facing hardships and challenges, Lauren remains strong and capable. She adapts to her circumstances and maintains a sense of determination and practicality in the face of adversity.

4. Empowered and proactive: Lauren takes control of her own life and actively work